In [1]:
from transformers import pipeline, AutoModel, AutoTokenizer, BertTokenizer, BertForQuestionAnswering
import transformers
import torch
import pandas as pd
%autosave 10

Autosaving every 10 seconds


In [43]:
DATA_DIRECTORY = "/home/xcs224u_student/notebooks/cs224u/data"

# baselines models

## BERT model trained on squad 2

In [35]:
#This model is BERT base uncased trained on SQuAD v2 
# https://huggingface.co/twmkn9/bert-base-uncased-squad2
BERT_MODEL = "twmkn9/bert-base-uncased-squad2"
qa_bert_pipeline = pipeline('question-answering', model=BERT_MODEL, tokenizer=BERT_MODEL)


In [36]:
qa_bert_pipeline(context="Normans", question="Who gave their name to Normandy in the 1000's and 1100's")



{'score': 0.9665267544103031, 'start': 0, 'end': 6, 'answer': 'Normans'}

In [37]:
qa_bert_pipeline(context="Normans", question="Who gave their name to Normandy in the 1000's and 1100's")



{'score': 0.9665267544103031, 'start': 0, 'end': 6, 'answer': 'Normans'}

## Baseline model using SciBERT - pretrained with SQUAD V2

In [31]:
# allenai/scibert_scivocab_uncased: scibert model fine-tuned on SQuAD V2 
# https://huggingface.co/ktrapeznikov/scibert_scivocab_uncased_squad_v2
SCIBERT_MODEL = "ktrapeznikov/scibert_scivocab_uncased_squad_v2"
qa_scibert_pipeline = pipeline('question-answering', model=SCIBERT_MODEL, tokenizer=SCIBERT_MODEL)


## Baseline model using BIOBERT - pretrained with SQUAD V2

In [40]:
# biobert model fine-tuned on SQuAD V2 
# https://huggingface.co/ktrapeznikov/biobert_v1.1_pubmed_squad_v2
BIOBERT_MODEL = "ktrapeznikov/biobert_v1.1_pubmed_squad_v2"
qa_biobert_pipeline = pipeline('question-answering', model=BIOBERT_MODEL, tokenizer=BIOBERT_MODEL)


# Qualitaive comparisons on covid-19

In [18]:
CONVID_ARTICLE_EASY="""Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).
The disease was first identified in December 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.
Common symptoms include fever, cough and shortness of breath.[5] Other symptoms may include fatigue, muscle pain, diarrhoea, sore throat, loss of smell and abdominal pain.[5][11][12] The time from exposure to onset of symptoms is typically around five days, but may range from two to fourteen days.[5][13] While the majority of cases result in mild symptoms, some progress to viral pneumonia and multi-organ failure.
As of 10 April 2020, more than 1.67 million[7] cases have been reported in more than 200 countries and territories,[15] resulting in more than 101,000 deaths.[7] More than 372,000 people have recovered. The virus is mainly spread between people during close contact,[a] often via small droplets produced during coughing,[b] sneezing, or talking.
While these droplets are produced when breathing out,
they usually fall to the ground or surfaces rather than being infectious over large distances.[6][19][20] People may also become infected by touching a contaminated surface and then their face.
The virus can survive on surfaces for up to 72 hours.
Coronavirus is most contagious during the first three days after onset of symptoms, although spread may be possible before symptoms
appear and in later stages of the disease.The standard method of diagnosis is by real-time reverse transcription polymerase chain reaction (rRT-PCR) from a nasopharyngeal swab.[23] The infection can also be diagnosed from a combination of symptoms, risk factors and a chest CT scan showing features of pneumonia.Recommended measures to prevent infection include frequent hand washing, maintaining physical distance from others (especially from those with symptoms), covering coughs and sneezes with a tissue or inner elbow and keeping unwashed hands away from the face.
The use of masks is recommended for those who suspect they have the virus and their caregivers.[28] Recommendations for mask use by the general public vary, with some authorities recommending against their use, some recommending their use and others requiring their use.
Currently, there is no vaccine or specific antiviral treatment for COVID-19.
Management involves treatment of symptoms, supportive care, isolation and experimental measures.The World Health Organization (WHO) declared the 2019–20 coronavirus outbreak a Public Health Emergency of International Concern (PHEIC)[33][34] on 30 January 2020 and a pandemic on 11 March 2020.[10] Local transmission of the disease has been recorded in many countries across all six WHO regions.[35]"""
CONVID_ARTICLE_MEDIUM1="""A familial cluster of 5 patients with COVID-19
pneumonia in Anyang, China, had contact before their symptom onset with an asymptomatic family member who had
traveled from the epidemic center of Wuhan. The sequence
of events suggests that the coronavirus may have been transmitted by the asymptomatic carrier. The incubation period
for patient 1 was 19 days, which is long but within the
reported range of 0 to 24 days.4 Her first RT-PCR result was
negative; false-negative results have been observed related to
the quality of the kit, the collected sample, or performance of
the test. RT-PCR has been widely deployed in diagnostic
virology and has yielded few false-positive outcomes.5 Thus,
her second RT-PCR result was unlikely to have been a falsepositive and was used to define infection with the coronavirus that causes COVID-19.
One previous study reported an asymptomatic 10-yearold boy with COVID-19 infection, but he had abnormalities
on chest CT.6 If the findings in this report of presumed transmission by an asymptomatic carrier are replicated, the prevention of COVID-19 infection would prove challenging.
The mechanism by which asymptomatic carriers could acquire and transmit the coronavirus that causes COVID-19
requires further study"""



## BERT SQUAD2

In [38]:
qa_bert_pipeline(context=CONVID_ARTICLE_EASY, question='What are the symptoms')

{'score': 0.5013612845677784,
 'start': 342,
 'end': 382,
 'answer': 'fever, cough and shortness of breath.[5]'}

In [39]:
qa_bert_pipeline(context=CONVID_ARTICLE_MEDIUM1, question='does RT-PCR work')

{'score': 0.09125602338691152,
 'start': 644,
 'end': 685,
 'answer': 'has yielded few false-positive outcomes.5'}

## SCIBERT SQUAD2

In [28]:
qa_scibert_pipeline(context=CONVID_ARTICLE_EASY, question='What are the symptoms')

{'score': 6.520916503848645e-05,
 'start': 667,
 'end': 734,
 'answer': 'symptoms, some progress to viral pneumonia and multi-organ failure.'}

In [32]:
qa_scibert_pipeline(context=CONVID_ARTICLE_MEDIUM1, question='does RT-PCR work')

{'score': 0.1103179816038562,
 'start': 644,
 'end': 685,
 'answer': 'has yielded few false-positive outcomes.5'}

## BIOBERT SQUAD2

In [41]:
qa_biobert_pipeline(context=CONVID_ARTICLE_EASY, question='What are the symptoms')

{'score': 0.4340334721582906,
 'start': 342,
 'end': 382,
 'answer': 'fever, cough and shortness of breath.[5]'}

In [42]:
qa_biobert_pipeline(context=CONVID_ARTICLE_MEDIUM1, question='does RT-PCR work')

{'score': 0.09305679627574825,
 'start': 585,
 'end': 639,
 'answer': 'RT-PCR has been widely deployed in diagnostic virology'}

# baseline metrics on SQUAD2:

## BERT base uncased SQUAD2

#### command:


python run_squad.py \
  --model_type bert \
  --model_name_or_path twmkn9/bert-base-uncased-squad2 \
   --version_2_with_negative \
  --do_lower_case \
  --do_eval \
  --predict_file  /data/home/xcs224u_student/notebooks/cs224u/data/squad/dev-v2.0.json\
  --max_seq_length 384 \
  --doc_stride 128 \
  --overwrite_cache \
  --output_dir /data/home/xcs224u_student/notebooks/cs224u/data/squad/pred_outputs/bert

{'exact': 72.3658721468879, 'f1': 75.83107045305708, 'total': 11873, 'HasAns_exact': 72.80701754385964, 'HasAns_f1': 79.74735146578041, 'HasAns_total': 5928, 'NoAns_exact': 71.9259882253995, 'NoAns_f1': 71.9259882253995, 'NoAns_total': 5945, 'best_exact': 72.3658721468879, 'best_exact_thresh': 0.0, 'best_f1': 75.8310704530571, 'best_f1_thresh': 0.0}

## biobert SQUAD2

### command: 

python run_squad.py \
  --model_type bert \
  --model_name_or_path ktrapeznikov/biobert_v1.1_pubmed_squad_v2 \
  --version_2_with_negative \
  --do_lower_case \
  --do_eval \
  --predict_file  /data/home/xcs224u_student/notebooks/cs224u/data/squad/dev-v2.0.json\
  --max_seq_length 384 \
  --doc_stride 128 \
  --overwrite_cache \
  --output_dir /data/home/xcs224u_student/notebooks/cs224u/data/squad/pred_outputs/biobert


{'exact': 72.72803840646846, 'f1': 76.63571518477706, 'total': 11873, 'HasAns_exact': 69.02834008097166, 'HasAns_f1': 76.85489986316776, 'HasAns_total': 5928, 'NoAns_exact': 76.41715727502103, 'NoAns_f1': 76.41715727502103, 'NoAns_total': 5945, 'best_exact': 72.72803840646846, 'best_exact_thresh': 0.0, 'best_f1': 76.63571518477703, 'best_f1_thresh': 0.0}

## scibert SQUAD2

### command:
python run_squad.py \
  --model_type bert \
  --model_name_or_path ktrapeznikov/scibert_scivocab_uncased_squad_v2 \
  --version_2_with_negative \
  --do_lower_case \
  --do_eval \
  --predict_file  /data/home/xcs224u_student/notebooks/cs224u/data/squad/dev-v2.0.json\
  --max_seq_length 384 \
  --doc_stride 128 \
  --overwrite_cache \
  --output_dir /data/home/xcs224u_student/notebooks/cs224u/data/squad/pred_outputs/scibert


{'exact': 75.06948538701255, 'f1': 78.48477186884588, 'total': 11873, 'HasAns_exact': 70.73211875843455, 'HasAns_f1': 77.57248589723473, 'HasAns_total': 5928, 'NoAns_exact': 79.39444911690497, 'NoAns_f1': 79.39444911690497, 'NoAns_total': 5945, 'best_exact': 75.07790785816559, 'best_exact_thresh': 0.0, 'best_f1': 78.49319433999901, 'best_f1_thresh': 0.0}

# Pre-Training